In [1]:
# Let's load and process the dataset
from fuel.datasets.dogs_vs_cats import DogsVsCats
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers.image import MinimumImageDimensions
from fuel.transformers import Flatten
from fuel.transformers import ScaleAndShift

# Load the training set
train = DogsVsCats(('train',), subset=slice(0, 20000))

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    train,
    iteration_scheme=ShuffledScheme(train.num_examples, 128)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512,512)

upscale_stream = MinimumImageDimensions(stream, (256,256),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
cropped_stream = RandomFixedSizeCrop(
    upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
default_cropped_stream = ScaleAndShift(cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
default_flattened_stream = Flatten(
    default_cropped_stream, which_sources=('image_features',))
    




In [ ]:
from fuel.server import start_server


start_server(default_flattened_stream)

In [2]:
iterator = default_flattened_stream.get_epoch_iterator()

import timeit
start_time = timeit.default_timer()
x, y = next(iterator)
total_time = timeit.default_timer() - start_time

In [3]:
total_time

0.47531447000801563

In [4]:
import os
import timeit

#os.environ['FUEL_DATA_PATH'] = os.path.abspath('./fuel_data')

# Let's load and process the dataset
from fuel.datasets.dogs_vs_cats import DogsVsCats
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers.image import MinimumImageDimensions
from fuel.transformers import Flatten
from fuel.transformers import ScaleAndShift


train_set = DogsVsCats(('train',), subset=slice(0, 20000))
valid_set = DogsVsCats(('train',), subset=slice(20000, 25000))
test_set = DogsVsCats(('test',))

batch_size = 128
n_train_batches = train_set.num_examples // batch_size

#################################################################################
# Train Stream
# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    train_set,
    iteration_scheme=ShuffledScheme(train_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512 x 512)

upscale_stream = MinimumImageDimensions(stream, (512,512),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
cropped_stream = RandomFixedSizeCrop(
    upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
default_cropped_stream = ScaleAndShift(cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
train_stream = Flatten(
    default_cropped_stream, which_sources=('image_features',))
    
######################################################################################
# Valid Stream

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    valid_set,
    iteration_scheme=ShuffledScheme(valid_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512,512)

valid_upscale_stream = MinimumImageDimensions(stream, (512,512),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
valid_cropped_stream = RandomFixedSizeCrop(
    valid_upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
valid_default_cropped_stream = ScaleAndShift(valid_cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
valid_stream = Flatten(
    valid_default_cropped_stream, which_sources=('image_features',))
    
##########################################################################################
# Test Stream 

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    test_set,
    iteration_scheme=ShuffledScheme(test_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512,512)

test_upscale_stream = MinimumImageDimensions(stream, (512,512),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
test_cropped_stream = RandomFixedSizeCrop(
    test_upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
test_default_cropped_stream = ScaleAndShift(test_cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
test_stream = Flatten(
    test_default_cropped_stream, which_sources=('image_features',))
    
####################################################################################################



In [6]:
from fuel.server import start_server




In [ ]:
start_server(train_stream, port = 5557)

In [ ]:
start_server(valid_stream, port = 5558)

In [ ]:
start_server(test_stream, port =5559)